In [1]:
from google.colab import files
uploaded = files.upload()


Saving brain_mri_unet_model.h5 to brain_mri_unet_model.h5
Saving Brain_tumor_classification(binary)(3).h5 to Brain_tumor_classification(binary)(3).h5


In [2]:
!pip install tensorflow opencv-python pillow scikit-learn


In [3]:

import tensorflow as tf
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt


In [4]:

class_model = tf.keras.models.load_model("Brain_tumor_classification(binary)(3).h5")
seg_model = tf.keras.models.load_model("brain_mri_unet_model.h5")


In [5]:

class_map = {0: "glioma", 1: "meningioma", 2: "no_tumor", 3: "pituitary"}


In [6]:


def preprocess_image(image_path):
    img = Image.open(image_path).convert('RGB')
    img = img.resize((224, 224))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array


In [17]:
class_map = {0: "glioma", 1: "meningioma", 2: "no_tumor", 3: "pituitary"}

def classify(image):
    img = image.convert('RGB').resize((224, 224))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = class_model.predict(img_array)
    class_index = int(np.argmax(prediction))
    confidence = float(np.max(prediction))
    class_name = class_map[class_index]

    return f"{class_name} ({confidence*100:.2f}%)"

def segment(image):
    img = image.convert("L").resize((256, 256))
    img_array = np.array(img).astype(np.float32) / 255.0
    img_array = np.expand_dims(img_array, axis=(0, -1))

    prediction = seg_model.predict(img_array)[0]
    mask = (prediction > 0.5).astype(np.uint8) * 255
    mask_img = Image.fromarray(mask.squeeze().astype(np.uint8))

    return mask_img



In [20]:
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 Brain Tumor Classifier & Segmenter")

    with gr.Row():
        image_input = gr.Image(type="pil", label="Upload Brain MRI")

    with gr.Row():
        classify_btn = gr.Button("🔍 Classify Tumor")
        segment_btn = gr.Button("🧪 Segment Tumor")
        reset_btn = gr.Button("🔄 Reset")

    with gr.Row():
        class_output = gr.Textbox(label="Classification Result")
        segment_output = gr.Image(label="Tumor Segmentation")

    classify_btn.click(fn=classify, inputs=image_input, outputs=class_output)
    segment_btn.click(fn=segment, inputs=image_input, outputs=segment_output)

    reset_btn.click(fn=lambda: (None, "", None), inputs=None, outputs=[image_input, class_output, segment_output])

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://015e9e5b419eebcac5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
